### Baseline Models

Simple If-Then Models

 - Home team always wins 
 
ML Models

 - LightGBM 
 - XGBoost
    

In [ ]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 500)

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb

print("XGB version:", xgb.__version__)

import lightgbm as lgb
from lightgbm import early_stopping
from lightgbm import log_evaluation

print("LGB version:", lgb.__version__)

from tqdm import tqdm

import shap

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path  # for Windows/Linux compatibility

DATAPATH = Path(r"../data")

In [ ]:
train = pd.read_csv(DATAPATH / "processed" / "train.csv")
test = pd.read_csv(DATAPATH / "processed" / "test.csv")

train.head()

**Model - Home team always wins**

In [ ]:
# train data

predict = np.ones((train.shape[0],))
true = train["TARGET"]

accuracy_score(true, predict), roc_auc_score(true, predict)

In [ ]:
# test data

predict = np.ones((test.shape[0],))
true = test["TARGET"]

accuracy_score(true, predict), roc_auc_score(true, predict)

### ML Baseline Models

**Fix Datatypes for smaller memory footprint**

In [ ]:
test["GAME_DATE_EST"]

In [ ]:
def fix_datatypes(df):

    if df.GAME_DATE_EST.dtype == "O":
        df["GAME_DATE_EST"] = df["GAME_DATE_EST"].str.split(" ").str[0]
    df["GAME_DATE_EST"] = pd.to_datetime(df["GAME_DATE_EST"])

    long_integer_fields = ["GAME_ID", "HOME_TEAM_ID", "VISITOR_TEAM_ID", "SEASON"]

    # convert long integer fields to int32 from int64
    for field in long_integer_fields:
        df[field] = df[field].astype("int32")

    # convert the remaining int64s to int8
    for field in df.select_dtypes(include=["int64"]).columns.tolist():
        df[field] = df[field].astype("int8")

    # convert float64s to float16s
    for field in df.select_dtypes(include=["float64"]).columns.tolist():
        df[field] = df[field].astype("float16")

    return df


train = fix_datatypes(train)
test = fix_datatypes(test)

**Basic Feature Engineering**

Because the basic data is post-game data, there would be data leakage if we used the data as is. We need to create features that are available before the game starts. Namely, we will do a rolling average for each stat for the last 5 games.

In [ ]:
def add_rolling_means(df, location):

    location_id = location + "_TEAM_ID"

    # sort games by the order in which they were played for each home or visitor team
    df = df.sort_values(
        by=[location_id, "GAME_DATE_EST"],
        axis=0,
        ascending=[
            True,
            True,
        ],
        ignore_index=True,
    )

    # rolling means
    feature_list = [
        "HOME_TEAM_WINS",
        "PTS_home",
        "FG_PCT_home",
        "FT_PCT_home",
        "FG3_PCT_home",
        "AST_home",
        "REB_home",
    ]

    if location == "VISITOR":
        feature_list = [
            "HOME_TEAM_WINS",
            "PTS_away",
            "FG_PCT_away",
            "FT_PCT_away",
            "FG3_PCT_away",
            "AST_away",
            "REB_away",
        ]

    roll_feature_list = []
    for feature in feature_list:
        roll_feature_name = location + "_" + feature + "_AVG_LAST_" + "5_" + location
        if feature == "HOME_TEAM_WINS":  # remove the "HOME_" for better readability
            roll_feature_name = (
                location + "_" + feature[5:] + "_AVG_LAST_" + "5_" + location
            )
        roll_feature_list.append(roll_feature_name)
        df[roll_feature_name] = (
            df.groupby(["HOME_TEAM_ID"])[feature]
            .rolling(5, closed="left")
            .mean()
            .values
        )

    return df


train = add_rolling_means(train, "HOME")
train = add_rolling_means(train, "VISITOR")
test = add_rolling_means(test, "HOME")
test = add_rolling_means(test, "VISITOR")

train

**Select Features**

In [ ]:
target = train["TARGET"]
test_target = test["TARGET"]

category_columns = [
    "HOME_TEAM_ID",
    "VISITOR_TEAM_ID",
    "SEASON",
    "HOME_TEAM_WINS",
    "PLAYOFF",
    "CONFERENCE_x",
    "CONFERENCE_y",
]

all_columns = train.columns.tolist()
drop_columns = [
    "TARGET",
    "GAME_DATE_EST",
    "GAME_ID",
]  # not really useful as-is

# non-rolling features, which would be data leakage
drop_columns1 = [
    "HOME_TEAM_WINS",
    "PTS_home",
    "FG_PCT_home",
    "FT_PCT_home",
    "FG3_PCT_home",
    "AST_home",
    "REB_home",
]
drop_columns2 = [
    "PTS_away",
    "FG_PCT_away",
    "FT_PCT_away",
    "FG3_PCT_away",
    "AST_away",
    "REB_away",
]

drop_columns = drop_columns + drop_columns1
drop_columns = drop_columns + drop_columns2

use_columns = [item for item in all_columns if item not in drop_columns]

train = train[use_columns]
test = test[use_columns]

**Options**

In [ ]:
K_FOLDS = 5
SEED = 13

### LightGBM


In [ ]:
%%time

NUM_BOOST_ROUND = 700
EARLY_STOPPING = 200
LOG_EVALUATION = 100

train_oof = np.zeros((train.shape[0],))
test_preds = 0
train_oof_shap = np.zeros((train.shape[0],train.shape[1]+1))
#train_oof_shap_interact = np.zeros((train.shape[0],train.shape[1]+1,train.shape[1]+1))
test_preds_shap = 0

lgb_params= {
            'seed': SEED,
            'verbose': 0,           
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': 'auc', 
            #'num_leaves': 31,
            #'learning_rate': 0.05,
            #'feature_fraction': 0.9,
            #'bagging_fraction': 0.8,
            #'bagging_freq': 5,

            }


# K-fold cross validation

kf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=SEED)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):
    
    train_df, val_df = train.iloc[train_ind], train.iloc[val_ind]
    train_target, val_target = target[train_ind], target[val_ind]

    train_lgbdataset = lgb.Dataset(train_df, label=train_target,)
    val_lgbdataset = lgb.Dataset(val_df, label=val_target, reference = train_lgbdataset )

    model =  lgb.train(lgb_params, 
                       train_lgbdataset,
                       valid_sets=val_lgbdataset,
                       num_boost_round = NUM_BOOST_ROUND,
                       callbacks=[log_evaluation(LOG_EVALUATION),early_stopping(EARLY_STOPPING,verbose=False)],
                       #verbose_eval= VERBOSE_EVAL,
                      )

    temp_oof = model.predict(val_df)
    temp_oof_shap = model.predict(val_df, pred_contrib=True)
    
    temp_test = model.predict(test)
    temp_test_shap = model.predict(test, pred_contrib=True)

    train_oof[val_ind] = temp_oof
    test_preds += temp_test/K_FOLDS

    train_oof_shap[val_ind, :] = temp_oof_shap
    test_preds_shap += temp_test_shap/K_FOLDS
    

    #for accuracy score, prediction probabilities must be convert to binary scores (Win or Lose)
    #determine optimum threshold for conveting probablities using ROC curve
    #generally 0.5 works for balanced data
    #fpr = false positive rate, tpr = true postive rate
    fpr, tpr, thresholds = roc_curve(val_target,temp_oof)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    temp_oof_binary = (temp_oof > optimal_threshold).astype(int)

    print(accuracy_score(val_target, temp_oof_binary), roc_auc_score(val_target, temp_oof))
    

    
# Out-of-Fold composite for train data

fpr, tpr, thresholds = roc_curve(target,train_oof)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
train_oof_binary = (train_oof > optimal_threshold).astype(int)

print()
print("Composite Train OOF CV Scores:")
print()
print("Accuracy Score:",accuracy_score(target, train_oof_binary))
print("AUC Score:", roc_auc_score(target, train_oof))
print("Optimal Threshold:", optimal_threshold)

#scores for Test data

test_preds_binary = (test_preds > optimal_threshold).astype(int)
print()
print("Test data Scores:")
print()
print("Accuracy Score:",accuracy_score(test_target, test_preds_binary))
print("AUC Score:", roc_auc_score(test_target, test_preds))





**Feature Importance via Split - the number of times a feature is used in the model**

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
lgb.plot_importance(model, importance_type="split", ax=ax)

**Feature Importance via Gain - the average gain of splits which use the feature**

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
lgb.plot_importance(model, importance_type="gain", ax=ax)

**Feature Importance via Shapley values**

In [ ]:
# summarize the effects of all the features
shap.summary_plot(train_oof_shap[:, :-1], train)

In [ ]:
shap.summary_plot(train_oof_shap[:, :-1], train[use_columns], plot_type="bar")

### XGBoost


In [ ]:
%%time

NUM_BOOST_ROUND = 700

train_oof = np.zeros((train.shape[0],))
test_preds = 0
train_oof_shap = np.zeros((train.shape[0],train.shape[1]+1))
train_oof_shap_interact = np.zeros((train.shape[0],train.shape[1]+1,train.shape[1]+1))
test_preds_shap = 0

xgb_params= {
            'seed': SEED,
            'eval_metric': 'auc',
             #'max_bin': 168, 
             #'max_depth': 1, #16
             #'alpha': 6.956941489832698, 
             #'gamma': 0.6029881527116713, 
             #'reg_lambda': 2.527966510426255, 
             #'colsample_bytree': 0.9087064850010729, 
             #'subsample': 0.31410604106509005, 
             #'min_child_weight': 7.877326540625619,
             #'num_parallel_tree' : 10,
             #'learning_rate': 0.03,  
            }

# K-fold cross validation

test_dmatrix = xgb.DMatrix(test)

kf = StratifiedKFold(n_splits=K_FOLDS, shuffle=True, random_state=SEED)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):
    
    train_df, val_df = train.iloc[train_ind], train.iloc[val_ind]
    train_target, val_target = target[train_ind], target[val_ind]

    train_dmatrix = xgb.DMatrix(train_df, label=train_target)
    val_dmatrix = xgb.DMatrix(val_df, label=val_target)

    model =  xgb.train(xgb_params, 
                       train_dmatrix, 
                       num_boost_round = NUM_BOOST_ROUND,
                      )

    temp_oof = model.predict(val_dmatrix)
    temp_oof_shap = model.predict(val_dmatrix, pred_contribs=True)
    temp_oof_shap_interact = model.predict(val_dmatrix, pred_interactions=True)
    
    temp_test = model.predict(test_dmatrix)
    temp_test_shap = model.predict(test_dmatrix, pred_contribs=True)

    train_oof[val_ind] = temp_oof
    test_preds += temp_test/K_FOLDS

    train_oof_shap[val_ind, :] = temp_oof_shap
    train_oof_shap_interact[val_ind, :,:] = temp_oof_shap_interact
    test_preds_shap += temp_test_shap/K_FOLDS
    
    #for accuracy score, prediction probabilities must be convert to binary scores (Win or Lose)
    #determine optimum threshold for conveting probablities using ROC curve
    #generally 0.5 works for balanced data
    #fpr = false positive rate, tpr = true postive rate
    fpr, tpr, thresholds = roc_curve(val_target,temp_oof)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    temp_oof_binary = (temp_oof > optimal_threshold).astype(int)

    print(accuracy_score(val_target, temp_oof_binary), roc_auc_score(val_target, temp_oof))
    

    
# Out-of-Fold composite for train data

fpr, tpr, thresholds = roc_curve(target,train_oof)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
train_oof_binary = (train_oof > optimal_threshold).astype(int)

print()
print("Composite Train OOF CV Scores:")
print()
print("Accuracy Score:",accuracy_score(target, train_oof_binary))
print("AUC Score:", roc_auc_score(target, train_oof))
print("Optimal Threshold:", optimal_threshold)

#scores for Test data

test_preds_binary = (test_preds > optimal_threshold).astype(int)
print()
print("Test data Scores:")
print()
print("Accuracy Score:",accuracy_score(test_target, test_preds_binary))
print("AUC Score:", roc_auc_score(test_target, test_preds))



**Feature Importance via Weight - the number of times a feature appears in a tree**

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
xgb.plot_importance(model, importance_type="weight", ax=ax)

**Feature Importance via Gain - the average gain of splits which use the feature**

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
xgb.plot_importance(model, importance_type="gain", ax=ax)

**Feature Importance via Shapley values**

In [ ]:
# summarize the effects of all the features
shap.summary_plot(train_oof_shap[:, :-1], train)

In [ ]:
shap.summary_plot(train_oof_shap[:, :-1], train[use_columns], plot_type="bar")